In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path =  "/content/drive/MyDrive/clef_data/data/clean_train.tsv"
test_path = "/content/drive/MyDrive/clef_data/data/1A_EnTest.tsv"
model_path = "/content/drive/MyDrive/clef_data/models/1A_en"

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 84 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 63.1 MB/s 
     |████████████████████████████████| 6.6 MB 66.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from collections import Counter
import random
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, f1_score, roc_curve, auc

In [ ]:
model_name = "roberta-large"
max_length = 64
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('DEVICE: ' + device)
model = model.to(device)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'clas

DEVICE: cuda


In [ ]:
column_n = ['topic', 'tweet_id', 'tweet_url', 'text', 'label']
remove_c = ['topic', 'tweet_id', 'tweet_url']

df_train = pd.read_csv(train_path, sep='\t')
df_test = pd.read_csv(test_path, sep='\t')
df_train = df_train.rename(columns={'class_label': 'label'})
df_test = df_test.rename(columns={'class_label': 'label'})
df_train = df_train.rename(columns={'tweet_text': 'text'})
df_test = df_test.rename(columns={'tweet_text': 'text'})

In [ ]:
def remove_unused_c(df,column_n=remove_c):
    df = df.drop(column_n,axis=1)
    return df

def clean_dataset(df):
    df = remove_unused_c(df)
    return df

def clean_text(text):
    text = str(text).replace(r'http[\w:\/\.]+', '@link')
    text = str(text).replace(r'\s\s+', ' ')
    return text

def nltk_preprocess(text):
    text = clean_text(text)
    return  text

In [ ]:
df_train = clean_dataset(df_train)
df_train["text"] = df_train.text.apply(nltk_preprocess)
df_test = clean_dataset(df_test)
df_test["text"] = df_test.text.apply(nltk_preprocess)


train_texts = df_train["text"].tolist()
valid_texts = df_test["text"].tolist()
train_labels = df_train["label"].tolist()
valid_labels = df_test["label"].tolist()

In [ ]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

In [ ]:
class TwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

train_dataset = TwitterDataset(train_encodings, train_labels)
valid_dataset = TwitterDataset(valid_encodings, valid_labels)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, pos_label=0)
    fpr, tpr, thresholds = roc_curve(labels, preds, pos_label=0)
    aucM = auc(fpr, tpr);
    #weighted_f1 = f1_score(labels, preds, average='weighted')
    print(classification_report(labels, preds))
    return {
        'accuracy': acc,
        'f1': f1,
        'auc': aucM,
        #'weighted_f1': weighted_f1,
        }

training_args = TrainingArguments(
    output_dir='trainer/results',
    num_train_epochs=15,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    disable_tqdm=False,
    load_best_model_at_end=True,
    warmup_ratio=0.06,
    weight_decay=0.01,
    logging_steps=8,
    learning_rate=3e-5,
    logging_dir='trainer/logs',
    run_name='roberta-classification',
    metric_for_best_model="f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2317
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1080


Epoch,Training Loss,Validation Loss,Accuracy,F1,Auc
0,0.484200,0.582645,0.562718,0.616794,0.304041
1,0.352600,0.397933,0.820557,0.889840,0.319406
2,0.259800,0.420110,0.818815,0.879070,0.218692
3,0.222200,0.595322,0.810105,0.881650,0.315138
4,0.186700,0.638606,0.827526,0.888639,0.246102
5,0.089400,1.195076,0.837979,0.900747,0.297161
6,0.037600,1.086046,0.832753,0.890160,0.217960
7,0.005400,1.358549,0.825784,0.888641,0.260988
8,0.000500,1.361072,0.843206,0.901099,0.252504
9,0.013700,1.556720,0.832753,0.896774,0.295027


***** Running Evaluation *****
  Num examples = 574
  Batch size = 32


              precision    recall  f1-score   support

           0       0.96      0.45      0.62       445
           1       0.33      0.94      0.49       129

    accuracy                           0.56       574
   macro avg       0.65      0.70      0.55       574
weighted avg       0.82      0.56      0.59       574



Saving model checkpoint to trainer/results/checkpoint-72
Configuration saved in trainer/results/checkpoint-72/config.json
Model weights saved in trainer/results/checkpoint-72/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 574
  Batch size = 32


...

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       445
           1       0.63      0.64      0.63       129

    accuracy                           0.83       574
   macro avg       0.76      0.76      0.76       574
weighted avg       0.83      0.83      0.83       574



Saving model checkpoint to trainer/results/checkpoint-1008
Configuration saved in trainer/results/checkpoint-1008/config.json
Model weights saved in trainer/results/checkpoint-1008/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 574
  Batch size = 32


              precision    recall  f1-score   support

           0       0.89      0.89      0.89       445
           1       0.63      0.63      0.63       129

    accuracy                           0.83       574
   macro avg       0.76      0.76      0.76       574
weighted avg       0.83      0.83      0.83       574



Saving model checkpoint to trainer/results/checkpoint-1080
Configuration saved in trainer/results/checkpoint-1080/config.json
Model weights saved in trainer/results/checkpoint-1080/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from trainer/results/checkpoint-648 (score: 0.901098901098901).


TrainOutput(global_step=1080, training_loss=0.10593944074832877, metrics={'train_runtime': 1775.3495, 'train_samples_per_second': 19.576, 'train_steps_per_second': 0.608, 'total_flos': 4047144928092672.0, 'train_loss': 0.10593944074832877, 'epoch': 14.99})